In [1]:
# testing implementation of n-dimensional inputs for turbo acquisition function

In [2]:
# imports
from dataclasses import dataclass
from typing import Any
import torch
import math
from botorch.generation import MaxPosteriorSampling
from botorch.models.gp_regression import ExactGP
from torch.quasirandom import SobolEngine
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.optim import optimize_acqf

In [3]:
# setup
dtype = torch.double
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import warnings
from botorch.exceptions import BadInitialCandidatesWarning
warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [64]:
# objects


def turbo_bounds_no_scaling(x_center, length):
    """get the bounds of the turbo trust region without scaling them by the lengthscales,
    this means that the region is a hyper-square, all sides the same length"""
    tr_lb = torch.clamp(x_center - length / 2.0, 0.0, 1.0)
    tr_ub = torch.clamp(x_center + length / 2.0, 0.0, 1.0)
    return tr_lb, tr_ub


@dataclass
class NewTurboState:
    """Class to maintain the trust region for TuRBO"""
    dim: int
    batch_size: int
    center: torch.Tensor
    lb: torch.Tensor = None
    ub: torch.Tensor = None
    train_x: Any = None
    train_y: Any = None
    length: float = 0.3
    length_min: float = 0.25 ** 3
    length_max: float = 0.5  # modifying for [0,1] interval from 1.6 to 0.5
    failure_counter: int = 0
    failure_tolerance: int = float("nan")  # post initialized
    success_counter: int = 0
    success_tolerance: int = 4  # original paper uses 3
    best_value: float = -float("inf")
    restart_triggered: bool = False
    domain_constraints: torch.Tensor = None

    def __post_init__(self):
        self.failure_tolerance = math.ceil(
            max([4.0 / self.batch_size, float(self.dim) / self.batch_size]))

    def get_training_samples_in_region(self):
        """query the training data and get the samples in the trust region - note this is for 1-d case"""
        # get the updated geometry of the trust region
        self.center = self.train_x[self.train_y.argmax(), :].clone()
        # self.lb = torch.clamp(self.center - self.length / 2.0, 0.0, 1.0)
        # self.ub = torch.clamp(self.center + self.length / 2.0, 0.0, 1.0)
        self.lb, self.ub = turbo_bounds_no_scaling(self.center, self.length)
        # get the indices of the evaluated data points in the trust region
        idx_below_ub = torch.where(self.train_x <= self.ub, True, False)
        idx_above_lb = torch.where(self.train_x >= self.lb, True, False)
        idx_in_tr = idx_below_ub & idx_above_lb
        # get the training points to use that are in the trust region
        # train_x_tr = self.train_x[idx_in_tr].unsqueeze(-1)
        # train_y_tr = self.train_y[idx_in_tr].unsqueeze(-1)
        train_x_tr = train_X[torch.all(idx_in_tr, dim=1), :]
        train_y_tr = train_Y[torch.all(idx_in_tr, dim=1)]
        # return the training points to use
        if train_x_tr.shape[0] < 1:
            print("\nNot enough points in tr, using global data to fit model\n")
            return self.train_x, self.train_y
            # depending on which {x, y} data are in current state, may need to revisit this
        else:
            return train_x_tr, train_y_tr


def new_update_state(state: NewTurboState, x_train, y_train, y_next):
    """Update the state of the trust region each iteration"""
    # check if the last iteration was successful and update attributes
    if torch.max(y_next) > state.best_value + 1e-3 * math.fabs(state.best_value):
        state.success_counter += 1
        state.failure_counter = 0
    else:
        state.success_counter = 0
        state.failure_counter += 1
    # modify trust region geometry based on success or failure of last step
    if state.success_counter == state.success_tolerance:  # expand trust region
        state.length = min(2.0 * state.length, state.length_max)
        state.success_counter = 0
    elif state.failure_counter == state.failure_tolerance:  # shrink the trust region
        state.length /= 2.0
        state.failure_counter = 0
    # update the best value seen
    state.best_value = max(state.best_value, torch.max(y_next).item())
    # check if the trust region needs to restart
    if state.length < state.length_min:
        state.restart_triggered = True
        state.length = 0.5  # assumes x in [0, 1]
        print("\nTuRBO restart triggered")
    # update training data set
    state.train_x = x_train
    state.train_y = y_train
    return state


def turbo_region_bounds(model: ExactGP, x_center, length, dim):
    """Get the bounds for the turbo trust region"""
    # scale the trust region to be proportional to the length scales
    weights = model.covar_module.base_kernel.lengthscale.squeeze().detach()
    weights = weights / weights.mean()
    if dim > 1:
        weights = weights / torch.prod(weights.pow(1.0 / len(weights)))
    tr_lb = torch.clamp(x_center - weights * length / 2.0, 0.0, 1.0)
    tr_ub = torch.clamp(x_center + weights * length / 2.0, 0.0, 1.0)
    return tr_lb, tr_ub


def turbo_thompson_sampling(state: NewTurboState, n_candidates, tr_lb, tr_ub, x_center, model, batch_size):
    """Convert candidates and trust region geometry to next sample point"""
    # thompson sampling
    dim = state.train_x.shape[-1]
    sobol = SobolEngine(dim, scramble=True)
    pert = sobol.draw(n_candidates).to(dtype=dtype, device=device)
    pert = tr_lb + (tr_ub - tr_lb) * pert
    # create perturbation mask
    prob_perturb = min(20.0 / dim, 1.0)
    mask = (
            torch.rand(n_candidates, dim, dtype=dtype, device=device)
            <= prob_perturb
    )
    ind = torch.where(mask.sum(dim=1) == 0)[0]
    if dim == 1:
        mask[ind, torch.randint(0, 1, size=(len(ind),), device=device)] = 1
    else:
        mask[ind, torch.randint(0, dim - 1, size=(len(ind),), device=device)] = 1
    # create candidate points from the perturbations and the mask
    x_cand = x_center.expand(n_candidates, dim).clone()
    x_cand[mask] = pert[mask]

    # sample the candidate points
    thompson_sampling = MaxPosteriorSampling(model=model, replacement=False)
    with torch.no_grad():
        return thompson_sampling(x_cand, num_samples=batch_size)


def generate_batch(state: NewTurboState,  # trust region state
                   model: ExactGP,  # GP model
                   dim,  # number of input dimensions
                   x,  # evaluated points on [0,1] for each dimension
                   y,  # evaluated function values corresponding to x
                   batch_size,
                   n_candidates=None,
                   num_restarts=10,
                   raw_samples=512,
                   acqf='ts',  # 'ei' or 'ts'
                   ):
    """Acquisition function for TuRBO, wraps Thompson sampling or Expected Improvement constrained to
    trust region boundaries"""
    assert acqf in ("ts", "ei")
    assert x.min() >= 0.0 and x.max() <= 1.0 and torch.all(torch.isfinite(y))
    if n_candidates is None:
        n_candidates = min(5000, max(2000, 200 * x.shape[-1]))

    # # scale the trust region to be proportional to the length scales
    x_center = x[y.argmax(), :].clone()
    tr_lb, tr_ub = turbo_region_bounds(model, x_center, state.length, dim=dim)

    if acqf == 'ts':
        x_next = turbo_thompson_sampling(state, n_candidates, tr_lb, tr_ub, x_center, model, batch_size)

    elif acqf == 'ei':
        ei = qExpectedImprovement(model, y.max(), maximize=True)
        x_next, acq_value = optimize_acqf(
            ei,
            bounds=torch.stack([tr_lb, tr_ub]),
            q=batch_size,
            num_restarts=num_restarts,
            raw_samples=raw_samples,
        )

    return x_next

In [47]:
# n-d test problem
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_mll

train_X = torch.rand(20, 3, dtype=dtype)
train_Y = torch.sin(train_X).sum(dim=1, keepdim=True)
model_example = SingleTaskGP(train_X, train_Y)
mll_example = ExactMarginalLogLikelihood(model_example.likelihood, model_example)
fitting = fit_gpytorch_mll(mll_example)

In [48]:
train_Y.argmax()

tensor(18)

In [49]:
test_lb, test_ub = turbo_region_bounds(model_example, train_X[5, :].clone(), 0.25, 2)
print(test_lb)
print(test_ub)

tensor([0.8058, 0.0661, 0.3534], dtype=torch.float64)
tensor([1.0000, 0.3054, 0.6081], dtype=torch.float64)


In [50]:
model_example.covar_module.base_kernel.lengthscale.squeeze().detach()

tensor([1.3371, 1.2474, 1.3278], dtype=torch.float64)

In [51]:
test_lb, test_ub = turbo_bounds_no_scaling(train_X[5, :].clone(), 0.25)
print(test_lb)
print(test_ub)

tensor([0.8090, 0.0608, 0.3557], dtype=torch.float64)
tensor([1.0000, 0.3108, 0.6057], dtype=torch.float64)


In [52]:
idx_below_ub = torch.where(train_X <= test_ub, True, False)
idx_above_lb = torch.where(train_X >= test_lb, True, False)

In [53]:
idx_in_tr = idx_below_ub & idx_above_lb
# get the training points to use that are in the trust region
train_x_tr = train_X[torch.all(idx_in_tr, dim=1), :]
train_y_tr = train_Y[torch.all(idx_in_tr, dim=1)]

In [61]:
train_x_tr.shape[0]

1

In [59]:
train_y_tr.shape

torch.Size([1, 1])

In [62]:
# test single iteration
train_X = torch.rand(20, 2, dtype=dtype)
train_Y = torch.sin(train_X).sum(dim=1, keepdim=True)
model_example = SingleTaskGP(train_X, train_Y)
mll_example = ExactMarginalLogLikelihood(model_example.likelihood, model_example)
fitting = fit_gpytorch_mll(mll_example)

In [66]:
test_state = NewTurboState(dim=2, batch_size=1, center=train_X[train_Y.argmax(), :].clone())
test_state = new_update_state(test_state, train_X, train_Y, train_Y.max())

In [68]:
N_CANDIDATES = min(5000, max(2000, 200 * train_X.shape[-1]))
NUM_RESTARTS = 10
RAW_SAMPLES = 512

new_x_turbo = generate_batch(
            state=test_state,
            model=model_example,
            dim=test_state.train_x.shape[-1],
            x=test_state.train_x,
            y=test_state.train_y,
            batch_size=1,
            n_candidates=N_CANDIDATES,
            num_restarts=NUM_RESTARTS,
            raw_samples=RAW_SAMPLES,
            acqf='ts',
        )

In [70]:
new_obj_turbo = torch.sin(new_x_turbo).sum(dim=1, keepdim=True)

In [71]:
train_x_turbo = torch.cat([train_X, new_x_turbo])
train_obj_turbo = torch.cat([train_Y, new_obj_turbo])

In [72]:
test_state = new_update_state(test_state, x_train=train_x_turbo, y_train=train_obj_turbo, y_next=new_obj_turbo)

In [73]:
tr_x_turbo, tr_obj_turbo = test_state.get_training_samples_in_region()

IndexError: The shape of the mask [21] at index 0 does not match the shape of the indexed tensor [20, 2] at index 0